# Period Module

## Initialisation

Import modules, etc

In [1]:
import os
import operator
import jinja2

from datetime import datetime

import unittest

from common import Printable, testExit, projdir

from fuzzy import FuzzyMatch
from entrant import Entrant

from constants import *

## Period Class

The period class contains common methods for events, sessions and courses

In [2]:
class Period(Printable):
    def __init__(self, parent=None, verbosity=1):
        '''Initialise period object'''

        super().__init__(verbosity=verbosity)

        self.period = self.__class__.__name__
        self.parent = parent

        if parent:
            self.year = parent.year
            self.entrants = parent.entrants
            self.names = parent.names
            self.sailNos = parent.sailNos
            self.gt31Ids = parent.gt31Ids
            self.fuzzyMatch = parent.fuzzyMatch
            self.appConfig = parent.appConfig
            self.eventConfig = parent.eventConfig
            self.reports = parent.reports

        else:
            self.year = None
            self.entrants = {}
            self.names = {}
            self.sailNos = {}
            self.gt31Ids = {}
            self.fuzzyMatch = FuzzyMatch()
            self.appConfig = None
            self.eventConfig = None
            self.reports = None

        self.runs = {}
        self.numRuns = 0


    def sortRuns(self):
        '''Sort runs for each person, fastest to slowest'''

        entrants = []

        for entrantId in self.runs:
            # Sort the runs for the individual entrant
            runs = self.runs[entrantId]
            runs.sort(key=lambda x: x.data[T_SPEED], reverse=True)
            
            # Period sort will be based on speed descending (hence the negative) and date/time ascending
            bestRun = runs[0]
            entrants.append((entrantId, runs, -bestRun.speed, bestRun.date, bestRun.time))

        # Sort avoids using a lambda function call
        entrants.sort(key = operator.itemgetter(2, 3, 4))
        
        # Re-create self.runs so that it is correctly sorted (Python 3.7 upwards)
        self.runs = {}
        for entrant in entrants:
            self.runs[entrant[0]] = entrant[1]


    def getEventName(self):
        '''Get the event name from event config or app config'''

        eventName = None

        if 'Event' in self.eventConfig:
            if 'Name' in self.eventConfig['Event']:
                eventName = self.eventConfig['Event']['Name']

        if eventName is None and 'Event' in self.appConfig:
            if 'Name' in self.appConfig['Event']:
                eventName = self.appConfig['Event']['Name']

        if eventName is None:
            eventName = 'Weymouth Speed Week'

        return eventName


    def runReports(self):
        '''Run all of the reports results'''

        numWarnings = 0

        # Note: It is slightly debatable whether this class should interact with the Reports class at such a low level
        className = self.__class__.__name__
        if className in self.reports.periods:
            reportSets = self.reports.periods[className]

            templatePath = os.path.join(projdir, 'python', 'templates')
            templateLoader = jinja2.FileSystemLoader(searchpath=templatePath)
            templateEnv = jinja2.Environment(loader=templateLoader, autoescape=True, trim_blocks=True, lstrip_blocks=True)

            eventName = self.getEventName()
            if className == 'Session':
                periodDate = datetime.strptime(self.date, "%Y%m%d").strftime("%a %-d %b")
                pageComment = self.comment
            else:
                periodDate = self.year
                pageComment = None

            for reportSet in reportSets.keys():
                reports = reportSets[reportSet]
                numResults = 0

                for report in reports:
                    report.prepareResults(self.runs)
                    numResults += len(report.results)

                    if report.warning and len(report.results) > 0:
                        numWarnings += 1

                htmlDir = os.path.join(projdir, 'docs', 'results', str(self.year))
                rootPath = '..'
                if className == 'Session':
                    htmlDir = os.path.join(htmlDir, self.date)
                    rootPath = os.path.join(rootPath, '..')
                if not os.path.exists(htmlDir):
                    os.makedirs(htmlDir)

                htmlFile = os.path.join(htmlDir, reportSet + '.html')

                if numResults > 0 or pageComment:
                    template = templateEnv.get_template("results.html")
                    html = template.render(
                                    eventName=eventName,
                                    className=className,
                                    pageTitle=reports[0].suptitle,
                                    pageComment=pageComment,
                                    pageDescription=reports[0].suptitle,
                                    periodDate=periodDate,
                                    cssPath=os.path.join(rootPath, 'css'),
                                    reports=reports)

                    with open(htmlFile, 'w', encoding='utf-8') as f:
                        f.write(html)
                else:
                    try:
                        os.remove(htmlFile)
                    except OSError:
                        pass
                    
        if numWarnings > 0:
            if className == 'Session':
                self.logWarning('{} report on {} generated results - {}'.format(
                    className, self.date, report.filter))
            else:
                self.logWarning('{} report generated results - {}'.format(
                    className, report.filter))

## Unit Tests

A handful of very basic tests, including a dummy session class

In [3]:
class TestPeriod(unittest.TestCase):
    '''Class to test Period class'''
    
    def testDummy(self, session=None):
        '''Test using dummy data'''

        pass

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [4]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## All Done!